In [11]:
# Dependencies
import pandas as pd
import requests
import time
import json
import operator as op
import numpy as np
from scipy.stats import linregress
from matplotlib import pyplot as plt

In [12]:
# Import cities file as DataFrame
data_df = pd.read_csv("athlete_events.csv")
data_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [13]:
# checking columns of dataframe
data_df.columns

Index(['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Team', 'NOC', 'Games',
       'Year', 'Season', 'City', 'Sport', 'Event', 'Medal'],
      dtype='object')

In [14]:
# Unique cities that held Olympic games
cities_held_games = data_df['City'].unique()
cities_held_games

array(['Barcelona', 'London', 'Antwerpen', 'Paris', 'Calgary',
       'Albertville', 'Lillehammer', 'Los Angeles', 'Salt Lake City',
       'Helsinki', 'Lake Placid', 'Sydney', 'Atlanta', 'Stockholm',
       'Sochi', 'Nagano', 'Torino', 'Beijing', 'Rio de Janeiro', 'Athina',
       'Squaw Valley', 'Innsbruck', 'Sarajevo', 'Mexico City', 'Munich',
       'Seoul', 'Berlin', 'Oslo', "Cortina d'Ampezzo", 'Melbourne',
       'Roma', 'Amsterdam', 'Montreal', 'Moskva', 'Tokyo', 'Vancouver',
       'Grenoble', 'Sapporo', 'Chamonix', 'St. Louis', 'Sankt Moritz',
       'Garmisch-Partenkirchen'], dtype=object)

In [16]:
summer_host_data = pd.read_csv("summer_host_cities.csv")
summer_host_data.head()

,Year,Host City,Country
0,1896,Athens,Greece
1,1900,Paris,France
2,1904,St. Louis,United States
3,1908,London,United Kingdom
4,1912,Stockholm,Sweden


In [18]:
winter_host_data = pd.read_csv("winter_host_cities.csv")
winter_host_data.head()

,Year,Host City,Country
0,1924,Chamonix,France
1,1928,St. Moritz,Switzerland
2,1932,Lake Placid,United States
3,1936,Garmisch-Partenkirchen,Germany
4,1940,Cancelled (WWII – Sapporo had been awarded),NaN
